In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [2]:
timestamp = '20241009-143022'

### <font color=blue> Compute GROUND TRUTH incident deaths for nyt county-level dataset.

In [3]:
fpath = os.path.join(DATA_DIR,'processed',f'formatted_nytimes-us-counties_{timestamp}.csv')
df_nyt = pd.read_csv(fpath)

## Compute inc deaths as (cum deaths) at t minus (cum deaths) at t-1 ...
## keep nans and negatives !!

df_nyt = df_nyt.sort_values(['Fips','Date'])

dfs = []
for fips in tqdm(df_nyt.Fips.unique()):
    
    df_county = df_nyt[df_nyt.Fips==fips]
    df_county.reset_index(inplace=True, drop=True)
    
    inc_deathss = []
    for i in range(len(df_county)):
        if i==0:
            inc_deaths = np.nan
        else:          
#             inc_deaths = df_county.True_cum_deaths.values[i] - df_county.True_cum_deaths.values[i-1]
            inc_deaths = df_county.at[i,'True_cum_deaths'] - df_county.at[i-1,'True_cum_deaths']   
        inc_deathss.append(inc_deaths)
            
    df_county['True_inc_deaths'] = inc_deathss
    dfs.append(df_county)
    
df_tot = pd.concat(dfs)
df_tot

100%|██████████| 3141/3141 [00:03<00:00, 980.33it/s] 


,Fips,Date,True_cum_deaths,True_inc_deaths
0,1001,20200613,6.0,NaN
1,1001,20200620,9.0,3.0
2,1001,20200627,12.0,3.0
3,1001,20200704,13.0,1.0
4,1001,20200711,15.0,2.0
...,...,...,...,...
95,78030,20220409,59.0,0.0
96,78030,20220416,59.0,0.0
97,78030,20220423,59.0,0.0
98,78030,20220430,59.0,0.0


### <font color=blue> Add necessary columns (for naive death computation) from acs and hub to main (nyt) dataframe.

In [5]:
fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_normed_pop_ratio_{timestamp}.csv')
df_acs = pd.read_csv(fpath)
fpath = os.path.join(DATA_DIR,'processed',f'formatted_COVIDhub-ensemble_{timestamp}.csv')
df_hub = pd.read_csv(fpath)

## Join with acs[[fips,pop,state_fips,state_pop,ratio]] ...
df = df_tot.merge(df_acs[['Fips','Pop','State_fips','State_pop','Ratio']], on='Fips')

## Add on proj_inc_deaths from hub ...
df = df.merge(df_hub,on=['State_fips','Date'])

## Reorder columns
df = df[['Fips','State_fips','Pop','State_pop','Ratio','Date',
         'Proj_inc_deaths', 'True_cum_deaths', 'True_inc_deaths']]
df

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,True_inc_deaths
0,1001,1,58239.0,4997675,0.011653,20200613,69.729674,6.0,NaN
1,1001,1,58239.0,4997675,0.011653,20200620,83.302156,9.0,3.0
2,1001,1,58239.0,4997675,0.011653,20200627,68.179357,12.0,3.0
3,1001,1,58239.0,4997675,0.011653,20200704,76.238975,13.0,1.0
4,1001,1,58239.0,4997675,0.011653,20200711,88.362530,15.0,2.0
...,...,...,...,...,...,...,...,...,...
309954,56045,56,6891.0,576641,0.011950,20220409,8.000000,18.0,0.0
309955,56045,56,6891.0,576641,0.011950,20220416,5.000000,18.0,0.0
309956,56045,56,6891.0,576641,0.011950,20220423,4.000000,18.0,0.0
309957,56045,56,6891.0,576641,0.011950,20220430,4.000000,18.0,0.0


### <font color=blue> Compute naive inc deaths.

In [6]:
df['Naive_inc_deaths'] = df.apply(lambda x: x.Proj_inc_deaths * x.Ratio, axis=1)
df

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,True_inc_deaths,Naive_inc_deaths
0,1001,1,58239.0,4997675,0.011653,20200613,69.729674,6.0,NaN,0.812575
1,1001,1,58239.0,4997675,0.011653,20200620,83.302156,9.0,3.0,0.970738
2,1001,1,58239.0,4997675,0.011653,20200627,68.179357,12.0,3.0,0.794509
3,1001,1,58239.0,4997675,0.011653,20200704,76.238975,13.0,1.0,0.888429
4,1001,1,58239.0,4997675,0.011653,20200711,88.362530,15.0,2.0,1.029708
...,...,...,...,...,...,...,...,...,...,...
309954,56045,56,6891.0,576641,0.011950,20220409,8.000000,18.0,0.0,0.095602
309955,56045,56,6891.0,576641,0.011950,20220416,5.000000,18.0,0.0,0.059751
309956,56045,56,6891.0,576641,0.011950,20220423,4.000000,18.0,0.0,0.047801
309957,56045,56,6891.0,576641,0.011950,20220430,4.000000,18.0,0.0,0.047801


### <font color=blue> Get cum deaths at t-1 for each sample.

In [7]:
from tqdm import tqdm

fipss = df.Fips.unique()

dfs = []
for fips in tqdm(fipss):
    df_county = df[df.Fips==fips]
    df_county.sort_values('Date',inplace=True)
    df_county.reset_index(inplace=True,drop=True)
    
    cum_deaths_tm1s = []
    
    for i in range(len(df_county)):
        if i==0:
            cum_deaths_tm1 = np.nan
        else:
            cum_deaths_tm1 = df_county.at[i-1, 'True_cum_deaths']
            
#         df_county.at[i, 'True_cum_deaths'] = cum_deaths_t_m1
        cum_deaths_tm1s.append(cum_deaths_tm1)
        
    df_county['Cum_deaths_tm1'] = cum_deaths_tm1s
        
    dfs.append(df_county)
        
#     break
        

100%|██████████| 3107/3107 [00:02<00:00, 1109.49it/s]


In [8]:
df_tot = pd.concat(dfs)

## Reorder columns 
df_tot = df_tot[['Fips', 'State_fips', 'Pop', 'State_pop', 'Ratio', 'Date',
                 'Proj_inc_deaths', 'True_cum_deaths',  'Cum_deaths_tm1',
                 'True_inc_deaths','Naive_inc_deaths']]

df_tot.reset_index(inplace=True,drop=True)
# df_tot
# df_tot

In [9]:
df = df_tot.copy()

## reset index
df.reset_index(inplace=True,drop=True)

## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch', f'processed_naive_deaths_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed', f'processed_naive_deaths_{DT}.csv')

df.to_csv(fpath,index=False)
print(DT)

## READ BACK AND CHECK
dff = pd.read_csv(fpath)
display(df, dff)
dff.dtypes

20241009-143422


,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,Naive_inc_deaths
0,1001,1,58239.0,4997675,0.011653,20200613,69.729674,6.0,NaN,NaN,0.812575
1,1001,1,58239.0,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,0.970738
2,1001,1,58239.0,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,0.794509
3,1001,1,58239.0,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,0.888429
4,1001,1,58239.0,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,1.029708
...,...,...,...,...,...,...,...,...,...,...,...
309954,56045,56,6891.0,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,0.095602
309955,56045,56,6891.0,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,0.059751
309956,56045,56,6891.0,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,0.047801
309957,56045,56,6891.0,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,0.047801


,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,Naive_inc_deaths
0,1001,1,58239.0,4997675,0.011653,20200613,69.729674,6.0,NaN,NaN,0.812575
1,1001,1,58239.0,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,0.970738
2,1001,1,58239.0,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,0.794509
3,1001,1,58239.0,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,0.888429
4,1001,1,58239.0,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,1.029708
...,...,...,...,...,...,...,...,...,...,...,...
309954,56045,56,6891.0,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,0.095602
309955,56045,56,6891.0,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,0.059751
309956,56045,56,6891.0,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,0.047801
309957,56045,56,6891.0,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,0.047801


Fips                  int64
State_fips            int64
Pop                 float64
State_pop             int64
Ratio               float64
Date                  int64
Proj_inc_deaths     float64
True_cum_deaths     float64
Cum_deaths_tm1      float64
True_inc_deaths     float64
Naive_inc_deaths    float64
dtype: object